In [4]:
import torch

In [5]:
a = torch.ones(3)

In [6]:
a

tensor([1., 1., 1.])

In [14]:
a[1]

tensor(1.)

In [15]:
float(a[1])

1.0

In [18]:
a[1].item()

1.0

In [19]:
type(a[1].item())

float

In [20]:
a[2] = 2.0

In [21]:
points = torch.tensor([[4.,1.],
                       [5.,3.],
                       [2.,1.]
                       ])

In [22]:
points.shape

torch.Size([3, 2])

In [25]:
points[1]

tensor([5., 3.])

In [26]:
torch.zeros(3,2)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [27]:
points = torch.tensor([[4.,1.],
                       [5.,3.],
                       [2.,1.]
                       ])

points[0,1]

tensor(1.)

In [28]:
points[0][1]

tensor(1.)

In [29]:
points[0].shape

torch.Size([2])

In [30]:
points[0]

tensor([4., 1.])

In [32]:
points[:,1]

tensor([1., 3., 1.])

In [33]:
# shape does not keep the geometry of the tensory, 
# i.e row vector vs column vector extracted from the original tensor
points[:,1].shape 

torch.Size([3])

In [34]:
some_list = list(range(6))
some_list

[0, 1, 2, 3, 4, 5]

In [36]:
some_list[0:4:2]

[0, 2]

In [38]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [39]:
points[1:, :]

tensor([[5., 3.],
        [2., 1.]])

In [40]:
img_t = torch.randn(3,5,5) # shape [channels (colors), rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # weights for combining color channels to get grey scale brightness (not trivial, aparently)

In [41]:
# imagine now that we have a batch of images, so the leading dimension is the batch size
# here we have 2 images
batch_t = torch.randn(2,3,5,5)

In [42]:
img_gray_naive = img_t.mean(-3) # take the mean for the third to last dimension, i.e. the color channel
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [46]:
''' 
We want to multiply the weights by the color channel values.
To do this, we need to unsqueeze the weights tensor, which means
it pads the tensor with an additional trailing dimension.

If we do img_t * weights the dimensions won't line up

'''
print(weights)
print(weights.unsqueeze(-1)) # this adds an extra dimension at dimension index -1, i.e an extra final (empty) dimension 
print(weights.unsqueeze(-1).unsqueeze(-1)) # same thing by then adds another empty dimension

tensor([0.2126, 0.7152, 0.0722])
tensor([[0.2126],
        [0.7152],
        [0.0722]])
tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]])


In [48]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1) # pad 2 extra trailing dimensions
img_weights = img_t * unsqueezed_weights
print(img_t)
print(img_weights)

tensor([[[-0.5994, -0.2495, -1.9543,  1.6787,  1.6019],
         [-1.2424,  1.2549,  0.1898, -0.0081, -0.5678],
         [-1.1053, -0.0828, -0.8733, -0.6413, -0.1430],
         [-0.2015,  0.0253,  1.8150,  0.9519, -1.5756],
         [ 0.4411, -1.5761,  2.0428, -0.3933, -0.7832]],

        [[-0.7239,  0.5119, -0.1204,  0.8924, -0.4763],
         [ 0.9363, -0.3296, -0.6300, -0.3760, -0.9687],
         [ 0.4823, -1.4348, -0.2831,  1.1522,  1.1197],
         [-0.5467,  0.3775, -0.7526, -0.2951,  0.4149],
         [-0.0986,  0.4345, -0.0718,  0.4520, -0.2145]],

        [[ 0.3985, -0.2885,  1.1508,  0.3642,  0.7306],
         [-0.1079, -0.5735,  0.4911,  0.3113,  1.5754],
         [-0.8418, -0.3111,  0.4578, -0.1271, -1.0427],
         [-0.2386,  1.3738, -0.4204,  2.1151,  1.6561],
         [ 0.8965,  1.4425,  0.3640,  2.0232,  0.2012]]])
tensor([[[-0.1274, -0.0531, -0.4155,  0.3569,  0.3406],
         [-0.2641,  0.2668,  0.0404, -0.0017, -0.1207],
         [-0.2350, -0.0176, -0.1857, -0.13

In [49]:
batch_weights = batch_t * unsqueezed_weights # note that broadcasting with torch will handing leading dimension (it starts alignment with trailing dimensions)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [52]:
# Now we can invoke Einstien summation (einsum), which is an idea copied from numpy
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights) # indicates the 3 dimensions can be labelled as c, h, & w. Broadcasting should leave h & w.
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [53]:
# Named Tensors (like a dictionary)
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])
weights_named

/var/folders/97/f9tvwl412dq2pq6wwlstvskh0000gp/T/ipykernel_80005/3180387998.py:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/TensorImpl.h:1938.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [54]:
img_named = img_t.refine_names(...,'channels','rows','columns') # the ellipsis (...) ignores all but the final dimensons
batch_named = batch_t.refine_names(...,'channels','rows','columns')
print(f"img named: {img_named.shape} {img_named.names}")
print(f"batch named: {batch_named.shape} {batch_named.names}")

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [55]:
# we can have torch pad the trailing and leading dimensions to ensure one tensor matches the dimenson of the other appropriately
weights_aligned = weights_named.align_as(img_named) # make weights_named match such that the channels columns align
weights_aligned.shape, weights_aligned.names 

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [56]:
gray_named = (img_named * weights_aligned).sum('channels') # weighted sum as before, but now more readable
gray_named.shape, gray_named.names # channels dimension gets summed away

(torch.Size([5, 5]), ('rows', 'columns'))

In [57]:
img_named[..., :3]

tensor([[[-0.5994, -0.2495, -1.9543],
         [-1.2424,  1.2549,  0.1898],
         [-1.1053, -0.0828, -0.8733],
         [-0.2015,  0.0253,  1.8150],
         [ 0.4411, -1.5761,  2.0428]],

        [[-0.7239,  0.5119, -0.1204],
         [ 0.9363, -0.3296, -0.6300],
         [ 0.4823, -1.4348, -0.2831],
         [-0.5467,  0.3775, -0.7526],
         [-0.0986,  0.4345, -0.0718]],

        [[ 0.3985, -0.2885,  1.1508],
         [-0.1079, -0.5735,  0.4911],
         [-0.8418, -0.3111,  0.4578],
         [-0.2386,  1.3738, -0.4204],
         [ 0.8965,  1.4425,  0.3640]]], names=('channels', 'rows', 'columns'))

In [58]:
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [59]:
img_named * weights_named

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [60]:
# It catches that we shouldn't be multiplying these two tensors (without proper alignment first, of course)

In [61]:
# to get back to unnamed tensors
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [62]:
gray_plain * weights_named # now the error is in the number of values per dimension

RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 1

In [63]:
# we ensure the last dimension takes the first 3 values, 
# meaning that we can now multiply the 3 valued weights_named
# by the final column
gray_plain[...,:3] * weights_named 

tensor([[-0.1310,  0.2090, -0.0302],
        [ 0.0846, -0.0074, -0.0271],
        [ 0.0105, -0.7626, -0.0256],
        [-0.0959,  0.2679, -0.0132],
        [ 0.0187,  0.0571,  0.0295]], names=(None, 'channels'))

In [64]:
gray_plain.names

(None, None)

In [65]:
# Now the named information is carried over. Of course
# this would be an incorrect operation. It just illustrates
# the point that using names would help ensure we are performing
# the operations that we think we are. 

In [66]:
# default dtype for values in a tensor is float = 32bit number
# you can use tensors to represent indexes of other tensors
# these are typically stored with int64 values

In [67]:
double_points = torch.ones(10,2, dtype = torch.double)
short_points = torch.tensor([[1,2],[3,4]], dtype=torch.short)

In [68]:
short_points.dtype

torch.int16

In [69]:
double_points = torch.zeros(10,2).double() # convert to double precision
short_points = torch.ones(10,2).short()

In [71]:
# more convenient method (I don't think so)
double_points = torch.zeros(10,2).to(torch.double)
short_points = torch.ones(10,2).to(torch.short)

In [72]:
# multiplying different types defaults to larger type
points_64 = torch.randn(5, dtype=torch.double)
points_short = points_64.short()
points_64 * points_short # notice the output dtype

tensor([0.0000, -0.0000, 1.0237, -0.0000, -0.0000], dtype=torch.float64)

In [73]:
# The Tensor API
# note it is an API because we are really calling some C functions, or applications
# to do the heavy but efficient lifting under the hood
a = torch.ones(3,2)
a_t = torch.transpose(a,0,1) # transport dimension 0 (rows) to dimension 1 (columns)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [74]:
torch.transpose?

Docstring:
transpose(input, dim0, dim1) -> Tensor

Returns a tensor that is a transposed version of :attr:`input`.
The given dimensions :attr:`dim0` and :attr:`dim1` are swapped.

If :attr:`input` is a strided tensor then the resulting :attr:`out`
tensor shares its underlying storage with the :attr:`input` tensor, so
changing the content of one would change the content of the other.

If :attr:`input` is a :ref:`sparse tensor <sparse-docs>` then the
resulting :attr:`out` tensor *does not* share the underlying storage
with the :attr:`input` tensor.

If :attr:`input` is a :ref:`sparse tensor <sparse-docs>` with compressed
layout (SparseCSR, SparseBSR, SparseCSC or SparseBSC) the arguments
:attr:`dim0` and :attr:`dim1` must be both batch dimensions, or must
both be sparse dimensions. The batch dimensions of a sparse tensor are the
dimensions preceding the sparse dimensions.

.. note::
    Transpositions which interchange the sparse dimensions of a `SparseCSR`
    or `SparseCSC` layout tens

In [75]:
a_t = a.transpose(0,1)

In [76]:
a_t

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [77]:
a_t.shape

torch.Size([2, 3])

In [78]:
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

I think it is interesting to note that both tensors are lists within lists, with the inner most list being the final dimension (here the data for some row, i.e. the column data). So, if we make 3 dimensional tensor we will have the final dimension be the length of the inner most lists.

Also, btw lists point to python objects all over memory. Numpy arrays ensure there is a block of memory stored for the entire 'list', and that the list is composed of a single data type that can be operated on efficiently and with special functions not applicable to lists. The same idea persists to pytorch, except with the addition of being able to push that all onto GPUs (and of course the deep learning library that comprises the other pillar of PyTorch)

In [80]:
torch.ones(2,3,4)

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [84]:
quick_tensor = torch.randn(3,3)
quick_tensor

tensor([[-1.5080, -1.3213,  1.5132],
        [ 0.9948,  0.3890,  0.9774],
        [-0.8030, -0.9537, -0.1519]])

In [85]:
quick_tensor.inverse() # no inverse method in numpy that I am aware of

tensor([[-0.6054,  1.1398,  1.3036],
        [ 0.4394, -1.0014, -2.0658],
        [ 0.4413,  0.2615, -0.5047]])

In [92]:
quick_tensor.storage()

/var/folders/97/f9tvwl412dq2pq6wwlstvskh0000gp/T/ipykernel_80005/1227932761.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  quick_tensor.storage()


 -1.508038878440857
 -1.3213382959365845
 1.513158917427063
 0.994814395904541
 0.38895902037620544
 0.9774173498153687
 -0.8030130863189697
 -0.9536927938461304
 -0.1519087851047516
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 9]

In [93]:
quick_tensor.untyped_storage()

 107
 7
 193
 191
 157
 33
 169
 191
 49
 175
 193
 63
 40
 172
 126
 63
 163
 37
 199
 62
 6
 56
 122
 63
 68
 146
 77
 191
 54
 37
 116
 191
 250
 141
 27
 190
[torch.storage.UntypedStorage(device=cpu) of size 36]

In [94]:
# trailing underscore for methods indicate the method is performed inplace
a = torch.ones(3,2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [101]:
''' 
I am skipping a lot about storage. It is interesting to note
that many methods applied to the tensor are just re-indexing
some meta data about how the tensor is stored. 

One main reason for the explanation of storage is that some methods
will not work on non-contiguous tensors, which are typically tensors
derived from other contiguous tenors; e.g. the transport of A is 
stored in the same location as A, just the meta-data updates. The
view method does not work with A^T. This methods is encounted in the 
next chapter.

Since I don't have CUDA cores, I am also skipping the to(device='cude')
methods. Another book I have has some code for pushing tensors and models
to Apple's M series chips' GPU. I will use that if/when necessary. 

Otherwise, this code may be nice to revisit if I get into cloud computing.

This discussion also leads into the idea of generalize tensors, which are 
really just subsets of tensors. For example, sparse tensors are stored in
the same way but with extra info regarding indexing so that we know where
the non-zero values actually are in the tensor. Much later (chapter 15),
we will meet quantized tensors. 
'''


" \nI am skipping a lot about storage. It is interesting to note\nthat many methods applied to the tensor are just re-indexing\nsome meta data about how the tensor is stored. \n\nOne main reason for the explanation of storage is that some methods\nwill not work on non-contiguous tensors, which are typically tensors\nderived from other contiguous tenors; e.g. the transport of A is \nstored in the same location as A, just the meta-data updates. The\nview method does not work with A^T. This methods is encounted in the \nnext chapter.\n\nSince I don't have CUDA cores, I am also skipping the to(device='cude')\nmethods. Another book I have has some code for pushing tensors and models\nto Apple's M series chips' GPU. I will use that if/when necessary. \n\nOtherwise, this code may be nice to revisit if I get into cloud computing.\n\nThis discussion also leads into the idea of generalize tensors, which are \nreally just subsets of tensors. For example, sparse tensors are stored in\nthe same way

In [121]:
a = torch.tensor(list(range(9)))
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [122]:
b = a.view(3,3) # should we not just do reshape?
b

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [123]:
a.storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]

In [124]:
b.storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]

In [125]:
a.storage() == b.storage()

False

In [126]:
a.cos()

tensor([ 1.0000,  0.5403, -0.4161, -0.9900, -0.6536,  0.2837,  0.9602,  0.7539,
        -0.1455])

In [127]:
a.cos_()

RuntimeError: result type Float can't be cast to the desired output type Long

In [128]:
a.short()
a.cos_()

RuntimeError: result type Float can't be cast to the desired output type Long

In [129]:
a.type()

'torch.LongTensor'

In [132]:
# what is happening here is that list(range(9)) created a list
# with values of dtype float64, i.e. long. PyTorch is expecting
# default values of dtype float32.
aa = a.float()
aa

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])

In [134]:
aa.cos_()

tensor([ 1.0000,  0.5403, -0.4161, -0.9900, -0.6536,  0.2837,  0.9602,  0.7539,
        -0.1455])